start docker with postgres

In [ ]:
# docker-compose up -d
# docker-compose down;rm -rf postgres-data

# docker exec -it bd_project_postgres_1 bash

In [1]:
! pip3 install psycopg2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import psycopg2

In [3]:
! pip3 install pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [4]:
import pandas as pd

In [5]:
def open_conn():
    conn = psycopg2.connect(dbname='postgres', 
                        user='postgres', 
                        password='postgres', 
                        host='localhost', 
                        port='5432') 
    cursor = conn.cursor()
    return conn, cursor


# посмотрим на таблицы

In [6]:
# cursor.execute('SELECT current_database();')


conn, cursor = open_conn()

cursor.execute("""SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
""")

tables = cursor.fetchall()
for t in tables:
    print("TABLE NAME: ", t[1])
    cursor.execute("""SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = %s;""", (t[1],))
    info = cursor.fetchall()
    print(pd.DataFrame(info))

cursor.close()
conn.close()

TABLE NAME:  country
              0                  1
0    country_id            integer
1  country_name  character varying
TABLE NAME:  store
          0                  1
0  store_id            integer
1      name  character varying
2   city_id            integer
TABLE NAME:  city
            0                  1
0     city_id            integer
1   city_name  character varying
2  country_id            integer
TABLE NAME:  sale
            0                            1
0     sale_id                      integer
1      amount                      numeric
2   date_sale  timestamp without time zone
3  product_id                      integer
4     user_id                      integer
5    store_id                      integer
TABLE NAME:  product
            0                  1
0  product_id            integer
1        name  character varying
TABLE NAME:  status_name
                0                  1
0  status_name_id            integer
1     status_name  character varying
TABLE 

# иллюстрация для ввода данных

1) вывожу последние 5 товаров

2) добавляю еще  товары

3) вывожу последние 5 товаров и вижу что все получилось

сгенеренные данные в таблице

In [7]:
conn, cursor = open_conn()

cursor.execute("""SELECT *
FROM product ORDER BY product_id DESC LIMIT 5;
""")

countries = cursor.fetchall()
print(pd.DataFrame(countries))

cursor.close()
conn.close()

    0           1
0  30  Product 30
1  29  Product 29
2  28  Product 28
3  27  Product 27
4  26  Product 26


еще три штуки 

In [10]:
conn, cursor = open_conn()
cursor.execute("INSERT INTO product (name) VALUES(%s)", ("BEST PRODUCT",))
cursor.execute("INSERT INTO product (name) VALUES(%s)", ("YET ANOTHER",))
cursor.execute("INSERT INTO product (name) VALUES(%s)", ("WORST OF ALL",))

conn.commit()

cursor.close()
conn.close()

они появились 

In [93]:
conn, cursor = open_conn()

cursor.execute("""SELECT *
FROM product ORDER BY product_id DESC LIMIT 5;
""")

countries = cursor.fetchall()
print(pd.DataFrame(countries))

cursor.close()
conn.close()

    0             1
0  31  BEST PRODUCT
1  30    Product 30
2  29    Product 29
3  28    Product 28
4  27    Product 27


# пример апдейта

триггер на апдейт в таблице order status поля status name id - меняется статус -> обновляется поле update_at

In [81]:
conn, cursor = open_conn()

cursor.execute(
"""CREATE OR REPLACE FUNCTION status_upd()
  RETURNS TRIGGER 
  LANGUAGE PLPGSQL
  AS
$$
BEGIN
	IF NEW.status_name_id <> OLD.status_name_id THEN
		 UPDATE order_status SET update_at = now() WHERE status_name_id = NEW.status_name_id;
	END IF;

	RETURN NEW;
END;
$$""")


cursor.execute("""CREATE TRIGGER status_upd
  AFTER UPDATE
  ON order_status
  FOR EACH ROW
  EXECUTE PROCEDURE status_upd();""")

conn.commit()

cursor.close()
conn.close()

In [82]:
conn, cursor = open_conn()

cursor.execute("""SELECT *
FROM order_status ORDER BY order_status_id DESC LIMIT 5;
""")

print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

     0                          1    2  3
0  200 2020-01-11 19:57:43.351428  200  4
1  199 2019-03-05 16:32:16.485399  199  3
2  198 2020-01-06 22:28:07.758725  198  5
3  197 2019-04-17 21:04:43.185830  197  2
4  196 2019-11-28 15:34:06.948986  196  3


update statusa

In [85]:
conn, cursor = open_conn()

# у id 196 (посл строка) status_name_id == 3. поменяю на 1
cursor.execute(
"""UPDATE order_status
SET status_name_id = 1 
WHERE order_status_id = 196;""")

cursor.execute("""SELECT *
FROM order_status ORDER BY order_status_id DESC LIMIT 5;
""")

print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

     0                          1    2  3
0  200 2020-01-11 19:57:43.351428  200  4
1  199 2019-03-05 16:32:16.485399  199  3
2  198 2020-01-06 22:28:07.758725  198  5
3  197 2019-04-17 21:04:43.185830  197  2
4  196 2021-12-24 12:26:10.438979  196  1


в последней строке поменялся таймстемп

#  SELECT с несколькими (от трех) таблицами в условии выборки 

возьму 3 таблицы

product, order status, sale


In [ ]:
# TABLE NAME:  product
#             0                  1
# 0  product_id            integer
# 1        name  character varying


# TABLE NAME:  sale
#             0                            1
# 0     sale_id                      integer
# 1      amount                      numeric
# 2   date_sale  timestamp without time zone
# 3  product_id                      integer
# 4     user_id                      integer
# 5    store_id                      integer

# TABLE NAME:  order_status
#                  0                            1
# 0  order_status_id                      integer
# 1        update_at  timestamp without time zone
# 2          sale_id                      integer
# 3   status_name_id                      integer

выведем продажи с именем продукта и статусом заказа, сумму заказа, и все отсотировнное по таймстемпу заказа - самые последние сверху

In [86]:
conn, cursor = open_conn()

cursor.execute("""SELECT sale.date_sale, sale.amount, pjoin.name, statusnamejoin.status_name 
FROM sale left join product pjoin using(product_id) 
left join order_status using(sale_id) 
left join status_name statusnamejoin using(status_name_id) 
ORDER BY date_sale DESC LIMIT 10;
""")

print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

                           0         1           2              3
0 2020-01-25 04:02:41.959237  4229.253  Product 17  Status Name 4
1 2020-01-23 19:37:31.044184  5995.307  Product 19  Status Name 2
2 2020-01-22 20:25:35.646844  9964.515  Product 28  Status Name 4
3 2020-01-18 13:20:33.497018  1022.764  Product 26  Status Name 4
4 2020-01-16 20:54:13.804815  5851.623  Product 24  Status Name 2
5 2020-01-15 23:07:13.786958  8844.789  Product 27  Status Name 2
6 2020-01-09 01:36:07.398691  3240.453  Product 19  Status Name 3
7 2020-01-08 13:43:30.257524  8819.007  Product 11  Status Name 4
8 2020-01-05 23:23:31.410946  3892.098   Product 7  Status Name 3
9 2020-01-03 23:33:56.117435  5766.241  Product 29  Status Name 4


# Вложенный SELECT


цена и названия товаров, которые дешевле среднего

In [87]:
conn, cursor = open_conn()

cursor.execute("""SELECT sale.amount, pjoin.name
FROM sale left join product pjoin using(product_id) 
where sale.amount > (SELECT AVG(amount) FROM sale)
ORDER BY sale.amount DESC LIMIT 10;
""")

print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

          0           1
0  9983.971  Product 23
1  9964.515  Product 28
2  9895.892  Product 25
3  9891.268  Product 21
4  9886.374   Product 4
5  9787.341  Product 14
6  9766.884   Product 9
7  9721.821   Product 6
8  9704.381   Product 2
9  9666.835  Product 27


в каком городе + стране успешнее всего продажи

In [108]:
# store, city, country

conn, cursor = open_conn()

cursor.execute("""SELECT SUM(sale.amount) as aggrsum, cityjoin.city_name, countryjoin.country_name
FROM sale left join store storejoin using(store_id) 
left join city cityjoin using(city_id) 
left join country countryjoin using(country_id) 
GROUP BY cityjoin.city_name, countryjoin.country_name ORDER BY aggrsum DESC LIMIT 10;
""")

print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

            0        1          2
0  245571.817   City 3  Country 1
1  192120.676   City 7  Country 1
2  153570.551   City 1  Country 4
3  147605.921   City 8  Country 2
4   89247.607   City 9  Country 2
5   68660.137   City 4  Country 3
6   64296.464   City 2  Country 1
7   43507.663   City 6  Country 4
8   12096.086  City 10  Country 2


топ тех, кто больше всех нам заплатил. аггрегации

sale, user

In [88]:
conn, cursor = open_conn()

cursor.execute("""SELECT SUM(sale.amount) as aggrsum, userjoin.name FROM sale 
left join users userjoin using(user_id) 
GROUP BY userjoin.name ORDER BY aggrsum DESC LIMIT 10;""")

print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

           0        1
0  41157.454  User 38
1  40691.812   User 7
2  38827.026  User 13
3  38800.132  User 49
4  36890.627   User 1
5  33668.162   User 8
6  32631.851   User 2
7  29248.804  User 16
8  29186.188  User 44
9  28928.729  User 48





удаление



решили убрать из ассортимента все товары которые никто ни разу не купил ))

In [98]:
conn, cursor = open_conn()

cursor.execute("""SELECT *
FROM product ORDER BY product_id DESC LIMIT 5;
""")

countries = cursor.fetchall()
print(pd.DataFrame(countries))

cursor.close()
conn.close()

    0           1
0  30  Product 30
1  29  Product 29
2  28  Product 28
3  27  Product 27
4  26  Product 26


In [109]:
# DROP TABLE IF EXISTS users; 

conn, cursor = open_conn()

cursor.execute("""select count(product_id) from product WHERE product_id NOT IN (SELECT product_id FROM sale);""")
print(pd.DataFrame(cursor.fetchall()))

cursor.execute("""DELETE FROM product WHERE product_id NOT IN (SELECT product_id FROM sale);""")

conn.commit()

# print(pd.DataFrame(cursor.fetchall()))

cursor.close()
conn.close()

   0
0  0


In [ ]:
conn, cursor = open_conn()

cursor.execute("""SELECT *
FROM product ORDER BY product_id DESC LIMIT 5;
""")

countries = cursor.fetchall()
print(pd.DataFrame(countries))

cursor.close()
conn.close()